In [34]:




import datetime
import pandas as pd
import os
import folium

# Get current date and time
now = datetime.datetime.now()


year = now.strftime("%Y")
month = now.strftime("%m")
day = now.strftime("%d")

dir_path = f'./data/real_time/{year}/{month}/{day}/'

# Get a list of all the csv files in the target directory

files = [f for f in os.listdir(dir_path) if f.endswith('.csv')]

# Initialize an empty dataframe
combined_df = pd.DataFrame()

# Loop through the files and read each into a dataframe
for file in files:
    file_path = os.path.join(dir_path, file)
    temp_df = pd.read_csv(file_path)

    # Append each dataframe to the combined_df
    combined_df = combined_df.append(temp_df, ignore_index=True)


/var/folders/8n/h7fgcf_96jbfwn6vj1l07p3h0000gn/T/ipykernel_10897/3898257807.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(temp_df, ignore_index=True)
/var/folders/8n/h7fgcf_96jbfwn6vj1l07p3h0000gn/T/ipykernel_10897/3898257807.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(temp_df, ignore_index=True)
/var/folders/8n/h7fgcf_96jbfwn6vj1l07p3h0000gn/T/ipykernel_10897/3898257807.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(temp_df, ignore_index=True)
/var/folders/8n/h7fgcf_96jbfwn6vj1l07p3h0000gn/T/ipykernel_10897/3898257807.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pa

In [35]:
combined_df['date_EST'] = pd.to_datetime(combined_df['date_EST'])

In [37]:
# create the map
m = folium.Map(location=[0, 0], zoom_start=2)

# group by 'aircraft.icao24' to gather all points related to the same flight
grouped = combined_df.sort_values('date_EST').groupby('aircraft.icao24')

for name, group in grouped:
    latitudes = group['geography.latitude']
    longitudes = group['geography.longitude']
    altitudes = group['geography.altitude']
    speeds = group['speed.horizontal']

    # Create a line for the entire flight path
    folium.PolyLine(locations=list(zip(latitudes, longitudes)), color="blue", weight=2.5, opacity=1).add_to(m)

    # Adding markers for each point along the flight path with popup information
    for lat, lon, alt, spd in zip(latitudes, longitudes, altitudes, speeds):
        folium.Marker([lat, lon], icon=folium.Icon(icon='plane'), popup=f'Flight {name}, Altitude: {alt}m, Speed: {spd}km/h').add_to(m)

    # # Adding markers for the start and end of each flight
    # folium.Marker([latitudes.iloc[0], longitudes.iloc[0]], popup=f'Start of Flight {name}').add_to(m)
    # folium.Marker([latitudes.iloc[-1], longitudes.iloc[-1]], popup=f'End of Flight {name}').add_to(m)

    # Add marker for the departure airport
    departure_lat = group['departure.latitudeAirport'].iloc[0]
    departure_lon = group['departure.longitudeAirport'].iloc[0]
    folium.Marker([departure_lat, departure_lon], icon=folium.Icon(color="green"), popup=f'Departure Airport for Flight {name}').add_to(m)

# Display the map
m